In [10]:
#regular CLSP example
from gurobipy import *
import time

start = time.time()

m = Model("CLSP")
#Parameters
demand = [[0,11,96,75,102,98,104,84,118,89, 120,109,91,104,107,83,90,97,93,113],
         [118,0,0,0,113,120,87,99,102,77,95,112,87,101,94,104,124,81,115,98],
         [75,97,0,0,104,77,102,123,124,109,95,120,103,99,87,107,111,94,84,110],
         [0,94,0,0,94,97,113,109,115,88,89,93,122,84,77,109,78,124,99,94],
         [76,90,0,117,112,76,107,123,99,98,78,109,96,78,79,117,76,122,124,95],
         [101,0,97,105,85,88,112,110,81,85,117,80,87,104,124,78,101,91,114,115],
         [0,117,119,98,107,96,89,104,106,108,78,120,78,120,112,107,110,100,105,108],
         [0,0,0,0,105,119,108,97,85,112,103,76,116,85,94,86,117,106,81,118],
         [0,77,0,121,79,107,123,102,105,84,100,120,103,117,118,78,75,97,124,81],
         [0,0,0,87,95,115,92,106,96,98,102,112,90,116,118,90,88,113,85,113]]
capacity = [1233,1233,1233,1233,1233,1233,1233,1233,1233,1233,1233,1233,1233,1233,1233,1233,1233,1233,1233,1233,]
hc = [1.2,1,1.2,1.2,0.8,1.1,1.2,1,1.1,1.1]
sc = [800,600,1000,600,600,1200,400,1200,600,600]
st = [17,5,11,11,11,8,17,11,8,5]
a = [1,1,1,1,1,1,1,1,1,1]
I0 = [0,0,0,0,0,0,0,0,0,0]

#Range of products and periods
NbProducts= range(len(sc)) #10
NbPeriods = range(len(capacity)) #20

#Decision variables
y = m.addVars(NbProducts, NbPeriods, vtype=GRB.BINARY, name="y")
I = m.addVars(NbProducts, NbPeriods, vtype=GRB.CONTINUOUS, name="I")
x = m.addVars(NbProducts, NbPeriods, vtype=GRB.CONTINUOUS, name="x")

#Objective
obj= quicksum(sc[i]*y[i,t]+hc[i]*I[i,t] for i in NbProducts for t in NbPeriods)
m.setObjective(obj, GRB.MINIMIZE)

#Capacity constraints
m.addConstrs(
    (quicksum(a[i]*x[i,t]+st[i]*y[i,t] for i in NbProducts) <= capacity[t]
        for t in NbPeriods), "Capacity-Ct")

#Inventory balance constraints
for i in NbProducts:
    for t in NbPeriods:
        if t==0:
            m.addConstr(I[i,t]==I0[i]+x[i,t]-demand[i][t])
        else:
            m.addConstr(I[i,t]==I[i,t-1]+x[i,t]-demand[i][t])

#Big M constraints
m.addConstrs((x[i,t]<= sum(demand[i][t:])*y[i,t] for i in NbProducts for t in NbPeriods), "BigM-Ct")
#alternative apprach:
#for i in NbProducts:
    #for t in NbPeriods
    #m.addGenConstrIndicator(y[i,t], False, x[i,t]==0.0)

#Set time limit
m.Params.timeLimit =60.0

m.optimize()

m.printAttr('x','y*')
print("Total time:", time.time()-start)

Set parameter TimeLimit to value 60
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i5-8250U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 420 rows, 600 columns and 1390 nonzeros
Model fingerprint: 0xcd634bad
Variable types: 400 continuous, 200 integer (200 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [8e-01, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 1e+03]
Found heuristic solution: objective 95148.700000
Presolve removed 40 rows and 49 columns
Presolve time: 0.01s
Presolved: 380 rows, 551 columns, 1278 nonzeros
Variable types: 355 continuous, 196 integer (196 binary)

Root relaxation: objective 2.739172e+04, 228 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd  

In [43]:
#CLSP-Rmin example
from gurobipy import *
import time

start = time.time()

m = Model("CLSP-Rmin")
#Parameters
d= [[258, 248, 193, 213, 238, 276, 329, 359, 375, 362],
    [341, 322, 251, 288, 330, 376, 443, 476, 498, 471],
    [510, 473, 390, 422, 482, 570, 667, 712, 753, 721],
    [508, 480, 390, 418, 478, 580, 653, 726, 744, 712]]
f=   [110, 120, 100, 120]
h= [1.5, 2, 2.5, 1.25]
hR= [1.5, 2, 2.5, 1.25]
tp= [1, 1, 1, 1]
tpR= [0.75, 0.75, 0.75, 0.75]
ts= [25, 20, 20, 15]
tsR= [25, 20, 20, 15]
b= [2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000]
k= [0, 0, 0, 0]
#O= [0,0,0,0,0,0,0,0,0,0]
O= [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01] #proportion of defective items
M= 100000
s0 = [0,0,0,0]
sR0 = [0,0,0,0]
T=len(b)
#print(T)
#Range of products and periods
Pro= range(len(f))
#print(Pro)
Pe = range(len(b))

#Decision variables default settings are 0.0 lb for all continous variables
x = m.addVars(Pro, Pe, vtype=GRB.BINARY, name="x")
s = m.addVars(Pro, Pe, vtype=GRB.INTEGER, name="s")
y = m.addVars(Pro, Pe, vtype=GRB.INTEGER, name="y")
yS = m.addVars(Pro, Pe, vtype=GRB.INTEGER, name="yS")
yR = m.addVars(Pro, Pe, vtype=GRB.INTEGER, name="yR")
sR = m.addVars(Pro, Pe, vtype=GRB.INTEGER, name="sR")
xR = m.addVars(Pro, Pe, vtype=GRB.BINARY, name="xR")
R = m.addVars(Pro, Pe, vtype=GRB.INTEGER, name="R")
m.update()

#Objective
obj= quicksum(((f[j]*x[j,t]+h[j]*s[j,t])+(f[j]*xR[j,t]+hR[j]*sR[j,t])) for j in Pro for t in Pe)
m.setObjective(obj, GRB.MINIMIZE) 
m.update()

#Capacity constraints
m.addConstrs(
    (quicksum(((tp[j]*y[j,t]+ts[j]*x[j,t])+(tpR[j]*yR[j,t]+tsR[j]*xR[j,t])) for j in Pro) <= b[t]
        for t in Pe), "Capacity-Ct")

#!production amount at least as size of lot size
for j in Pro:
    for t in Pe:
        m.addConstr(y[j,t]>=k[j]*x[j,t])
        m.addConstr(yS[j,t] ==(y[j,t]*(1-O[t])))
        m.addConstr(R[j,t]==(y[j,t]*O[t]))
    
#Inventory balance constraints
for j in Pro:
    for t in Pe:
        if t==0:
            m.addConstr(s[j,t]==0)#s0[j]+yS[j,t]+yR[j,t]-d[j][t])
        else:
            m.addConstr(s[j,t]==s[j,t-1]+yS[j,t]+yR[j,t]-d[j][t])
m.update()
for j in Pro:
    for t in Pe:
        if t==0:
            m.addConstr(sR[j,t]==0)#sR0[j]+R[j,t]-yR[j,t])
        else:
            m.addConstr(sR[j,t]==sR[j,t-1]+R[j,t]-yR[j,t])

m.update()
#s[j,T] the last period of product j equals 0
m.addConstrs((s[j,T-1] == 0) for j in Pro)    #T is defined as T=len(b) to count how many periods we have =10                 
m.addConstrs((sR[j,T-1] == 0) for j in Pro)
           
#alternative apprach:
#Big M+ Big M for rework
m.addConstrs((y[j,t]<= sum(d[j][t:])*x[j,t] for j in Pro for t in Pe), "BigM-Ct")
m.addConstrs((yR[j,t]<= sum(d[j][t:])*xR[j,t] for j in Pro for t in Pe), "BigM-Ct for Rework")
#alternative apprach:
#for i in NbProducts:
    #for t in NbPeriods
    #m.addGenConstrIndicator(y[i,t], False, x[i,t]==0.0)

#Set time limit
#m.Params.timeLimit =60.0
logging.basicConfig(filename='info.log', level=logging.INFO)

m.optimize()
#m.computeIIS()

#m.write('model.ilp')
m.printAttr('x','y*')
m.printAttr('x','s*')
m.update()

#holding cost, cost for rework and setup cost chronologically
pp=0
for j in Pro:
    for t in Pe:
        pp= pp + (h[j]*s[j,t])
ppp=0
for j in Pro:
    for t in Pe:
        ppp= ppp + (hR[j]*sR[j,t])
pppp=0
for j in Pro:
    for t in Pe:
        pppp=pppp+(f[j]*x[j,t]+f[j]*xR[j,t])
        
print(pp.getValue())
print(ppp.getValue())
print(pppp.getValue())
#print(ppp)
#print("Total time:", time.time()-start)

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i5-8250U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 298 rows, 320 columns and 688 nonzeros
Model fingerprint: 0xc6f822a6
Variable types: 0 continuous, 320 integer (80 binary)
Coefficient statistics:
  Matrix range     [1e-02, 6e+03]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+02, 2e+03]
Presolve removed 164 rows and 192 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -

    Variable            x 
-------------------------


GurobiError: Unable to retrieve attribute 'x'